# Sep 21, 2025: canonical systems

conda env: gt

In [1]:
import os
import nibabel as nib
import numpy as np
import pandas as pd

In [2]:
class ARGS():
    pass

args = ARGS()

args.SEED = 100

def set_seed(args):
    # gt.seed_rng(args.SEED)
    np.random.seed(args.SEED)

set_seed(args)

In [3]:
args.source = 'allen'
args.space = 'ccfv2'
args.brain_div = 'whl'
args.num_rois = 172
args.resolution = 200

PARC_DESC = (
    f'source-{args.source}'
    f'_space-{args.space}'
    f'_braindiv-{args.brain_div}'
    f'_nrois-{args.num_rois}'
    f'_res-{args.resolution}'
)
PARC_DESC

'source-allen_space-ccfv2_braindiv-whl_nrois-172_res-200'

In [4]:
args.GRAPH_DEF = f'constructed'
args.GRAPH_METHOD = f'pearson'
args.THRESHOLD = f'signed'
args.EDGE_DEF = f'binary'
args.EDGE_DENSITY = 20
args.LAYER_DEF = f'individual'
args.DATA_UNIT = f'grp'

BASE_path = f'{os.environ["HOME"]}/new_mouse_dataset'
PARCELS_path = f'{BASE_path}/parcels'
IN_ICS_path = f'{BASE_path}/joanes_rsns'
SYS_path = f'{BASE_path}/canonical_systems/joanes/space-{args.space}'
os.makedirs(f'{SYS_path}', exist_ok=True)
RSN_path = f'{SYS_path}/rsns'
os.makedirs(f'{RSN_path}', exist_ok=True)

In [5]:
parcels_img = nib.load(f'{PARCELS_path}/{PARC_DESC}_desc-parcels.nii.gz')
parcels_vol = parcels_img.get_fdata()
parcels_vol = np.round(parcels_vol, decimals=0)
rois_df = pd.read_csv(f'{PARCELS_path}/{PARC_DESC}_desc-names.csv')
roi_labels = rois_df['roi'].to_numpy()
roi_names = rois_df['name'].to_list()

In [6]:
# remove cerebellar and hind brain regions because we don't have ICs for those regions
excluded_rois = rois_df[rois_df['name'].str.contains('Pons|Medulla|Cerebellar', case=False, na=False,)]['roi'].to_list()

excluded_vol = np.where(np.isin(parcels_vol, excluded_rois), parcels_vol, 0)
excluded_mask = np.where(excluded_vol, 1, 0)

parcels_vol = np.where(np.isin(parcels_vol, excluded_rois), 0, parcels_vol)
parcels_mask = np.where(parcels_vol, 1, 0)

full ICs/both hemispheres

cover the brain with ICs

In [7]:
ICS_file = f'{IN_ICS_path}/RSNs_inAllenSpace.nii' 
#thanks Francesca for transforming Joanes's ICs into ccfv2 space!
ics_img = nib.load(ICS_file)
ics_vol = ics_img.get_fdata()

In [8]:
ics_vol.shape # x, y, z, ics

(57, 40, 66, 18)

In [9]:
thresh = 0
mask = np.abs(ics_vol) > thresh # threshold the ics
thresh_ics_vol = np.where(mask, ics_vol, -np.inf)

argmax_mask = np.argmax(thresh_ics_vol, axis=3) + 1
argmax_mask[np.all(~mask, axis=3)] = 0

argmax_mask *= parcels_mask # remove ICs from those regions

argmax_img = nib.Nifti1Image(
    argmax_mask.astype(np.int16),
    affine=ics_img.affine,
    header=ics_img.header,
)
nib.save(argmax_img, f'{SYS_path}/desc-ic-argmax-masks.nii.gz')

In [10]:
unique_ics = np.unique(argmax_mask)
unique_ics = unique_ics[unique_ics > 0] # remove background

ics_vol = np.zeros_like(ics_vol)
for idx_ic, ic in enumerate(unique_ics):
    ics_vol[..., idx_ic] = (argmax_mask == ic).astype(np.int16) # binary mask
    
ics_img = nib.Nifti1Image(
    ics_vol, 
    affine=argmax_img.affine,
    header=argmax_img.header,
)
nib.save(
    ics_img, 
    f'{SYS_path}/desc-ic-masks-combined.nii.gz'
)

In [11]:
IC_path = f'{SYS_path}/ics'
os.makedirs(f'{IC_path}', exist_ok=True)

for idx in range(ics_vol.shape[-1]):
    ic_vol = ics_vol[..., idx]
    ic_img = nib.Nifti1Image(
        ic_vol, 
        affine=ics_img.affine,
        header=ics_img.header,
    )
    nib.save(ic_img, f'{IC_path}/desc-{idx:02d}.nii.gz')

ics to rsns

In [12]:
df = []

df += [(
    0, 'Pir', 'olfactory'
)]
df += [(
    1, 'M1', 'sensory'
)]
df += [(
    2, f'S1 (vlp) + S2', 'somatosensory'
)]
df += [(
    3, 'S1 (hl + fl)', 'somatosensory'
)]
df += [(
    4, 'S1 (fl + bf)', 'somatosensory'
)]
df += [(
    5, 'S1 (bf)', 'somatosensory'
)]
df += [(
    6, 'V2 + RSD', 'sensory'
)]
df += [(
    7, 'Au (d + v)', 'sensory'
)]
df += [(
    8, 'Cg2', 'limbic'
)]
df += [(
    9, 'Cg1 + RS', 'limbic'
)]
df += [(
    10, 'HP (v)', 'limbic'
)]
df += [(
    11, 'HP (d)', 'limbic'
)]
df += [(
    12, 'CPu', 'basal ganglia'
)]
df += [(
    13, 'CPu + En', 'basal ganglia'
)]
df += [(
    14, 'Sep + Pal (v)', 'basal ganglia'
)]
df += [(
    15, 'Amyg', 'limbic'
)]
df += [(
    16, 'Th', 'limbic'
)]
df += [(
    17, 'MO + GI', 'olfactory'
)]
ics_df = pd.DataFrame(data=df, columns=['ic', 'name', 'rsn'])
ics_df.to_csv(f'{SYS_path}/desc-ic-names.csv')
ics_df

,ic,name,rsn
0,0,Pir,olfactory
1,1,M1,sensory
2,2,S1 (vlp) + S2,somatosensory
3,3,S1 (hl + fl),somatosensory
4,4,S1 (fl + bf),somatosensory
5,5,S1 (bf),somatosensory
6,6,V2 + RSD,sensory
7,7,Au (d + v),sensory
8,8,Cg2,limbic
9,9,Cg1 + RS,limbic


rsn masks

In [13]:
df = []

df += [(
    0, 'somatosensory', [2, 3, 4, 5],
)]
df += [(
    1, 'sensory', [1, 6, 7],
)]
df += [(
    2, 'olfactory', [0, 17],
)]
df += [(
    3, 'limbic', [9, 8, 10, 11, 15, 16],
)]
df += [(
    4, 'basal ganglia', [13, 12, 14],
)]
df += [(
    5, 'cerebellar', [],
)]
rsns_df = pd.DataFrame(data=df, columns=['rsn', 'name', 'ics'])
# rsns_df = rsns_df.sort_values(by='name').reset_index(drop=True)
rsns_df = rsns_df.reset_index(drop=True)
rsns_df.to_csv(f'{SYS_path}/desc-rsn-names.csv')
rsns_df

,rsn,name,ics
0,0,somatosensory,"[2, 3, 4, 5]"
1,1,sensory,"[1, 6, 7]"
2,2,olfactory,"[0, 17]"
3,3,limbic,"[9, 8, 10, 11, 15, 16]"
4,4,basal ganglia,"[13, 12, 14]"
5,5,cerebellar,[]


In [14]:
rsns_vol = np.zeros((*argmax_mask.shape, len(rsns_df)), dtype=np.int16)
for idx, row in rsns_df.iterrows():
    ics = row['ics']
    if len(ics) > 0:
        rsns_vol[..., idx] = np.sum(ics_vol[..., ics], axis=3)
    else:
        rsns_vol[..., idx] = excluded_mask

rsns_img = nib.Nifti1Image(
    rsns_vol, 
    affine=ics_img.affine,
    header=ics_img.header,
)
nib.save(rsns_img, f'{SYS_path}/desc-rsn-masks-combined.nii.gz')

In [15]:
for idx in range(rsns_vol.shape[-1]):
    rsn_vol = rsns_vol[..., idx]
    rsn_img = nib.Nifti1Image(
        rsn_vol,
        affine=rsns_img.affine,
        header=rsns_img.header,
    )
    nib.save(rsn_img, f'{RSN_path}/desc-{idx:02d}.nii.gz')